In [1]:
import pyspark
from pyspark.sql import SparkSession
print("Hello")

Hello


In [4]:
#Minio Configuration
s3_host = "minio"
s3_url = f"http://{s3_host}:9000"
s3_key = "minio"
s3_secret = "SU2orange!"
s3_bucket = "labe"
print(s3_url)

http://minio:9000


In [5]:
#Spark init
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-psyspark') \
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.1.2") \
    .config("spark.hadoop.fs.s3a.endpoint", s3_url) \
    .config("spark.hadoop.fs.s3a.access.key", s3_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret) \
    .config("spark.hadoop.fs.s3a.fast.upload", True) \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
sc = spark.sparkContext
print(s3_bucket)
sc.setLogLevel("ERROR")

labe


In [8]:
s3_bucket = "labe"
file_name = "syracuse-ny.csv"
print("Hello")
source = f"s3a://{s3_bucket}/{file_name}"
weather = spark.read.option("header", True).option("inferSchema", True)\
    .csv(source)
weather.show()

Hello
+---------+----------------+-----------------+----------------+--------------+--------------+-------------+------------+-------------+------------+------------------------+-------------------------+------------------------+-------------------+--------------------+-------------------+-----------------+------------------+-----------------+---------------+----------+-------------+--------------+
|      EST|Max TemperatureF|Mean TemperatureF|Min TemperatureF|Max Dew PointF|MeanDew PointF|Min DewpointF|Max Humidity|Mean Humidity|Min Humidity|Max Sea Level PressureIn|Mean Sea Level PressureIn|Min Sea Level PressureIn|Max VisibilityMiles|Mean VisibilityMiles|Min VisibilityMiles|Max Wind SpeedMPH|Mean Wind SpeedMPH|Max Gust SpeedMPH|PrecipitationIn|CloudCover|       Events|WindDirDegrees|
+---------+----------------+-----------------+----------------+--------------+--------------+-------------+------------+-------------+------------+------------------------+-------------------------+----

In [10]:
weather = spark.read.option("header", True).option("inferSchema", True)\
    .csv(source)
weather.createOrReplaceTempView("weather")

spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |  weather|       true|
+--------+---------+-----------+



In [15]:
query = """
WITH Source AS (
    SELECT 
        CAST(SPLIT(EST, '-')[0] AS INT) AS year, 
        CAST(SPLIT(EST, '-')[1] AS INT) AS month,
        CAST(`Min TemperatureF` AS INT) AS mintemp,
        CAST(`Max TemperatureF` AS INT) AS maxtemp
    FROM weather
)
SELECT year, month, AVG(mintemp) AS avgmintemp, AVG(maxtemp) AS avgmaxtemp
FROM Source
GROUP BY year, month
ORDER BY year, month
"""

spark.sql(query).printSchema()
weather2 = spark.sql(query)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- avgmintemp: double (nullable = true)
 |-- avgmaxtemp: double (nullable = true)



In [16]:
weather2.createOrReplaceTempView("monthly_syracuse_weather_averages")
query2 = '''
select * from monthly_syracuse_weather_averages
'''
spark.sql(query2).show()

+----+-----+------------------+------------------+
|year|month|        avgmintemp|        avgmaxtemp|
+----+-----+------------------+------------------+
|1997|    1|15.774193548387096| 31.64516129032258|
|1997|    2|22.607142857142858|37.785714285714285|
|1997|    3|25.032258064516128| 41.12903225806452|
|1997|    4| 34.43333333333333|              54.1|
|1997|    5|43.096774193548384| 61.58064516129032|
|1997|    6|              57.8|              78.4|
|1997|    7| 59.87096774193548| 80.19354838709677|
|1997|    8| 58.70967741935484| 78.38709677419355|
|1997|    9| 51.06666666666667|              69.4|
|1997|   10|38.935483870967744| 59.16129032258065|
|1997|   11|31.466666666666665|42.666666666666664|
|1997|   12|25.032258064516128|35.516129032258064|
|1998|    1|23.258064516129032| 35.41935483870968|
|1998|    2|24.142857142857142|              38.0|
|1998|    3|30.129032258064516|45.064516129032256|
|1998|    4|              38.0| 58.46666666666667|
|1998|    5| 52.45161290322581|